In [0]:
#/FileStore/tables/walmart_stock.csv
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Operations").getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/walmart_stock.csv',inferSchema=True,header=True,sep=",")

In [0]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [0]:
df.show(4)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 4 rows



In [0]:
#no null columns in dataframe
a=df.columns
for i in a:
    print(str(i),df.where(df[i].isNull()).count())

Date 0
Open 0
High 0
Low 0
Close 0
Volume 0
Adj Close 0


In [0]:
df.count()

Out[12]: 1258

In [0]:
#there are no duplicate rows
df.distinct().count()

Out[14]: 1258

In [0]:
#Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

In [0]:
df_new=df.withColumn(" HV Ratio",df["High"]/df["Volume"])
df_new.show(4)

+-------------------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|            HV Ratio|
+-------------------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
+-------------------+------------------+---------+---------+------------------+--------+---------

In [0]:
#2.	What day had the Peak High in Price?

In [0]:
df.orderBy(df['High'].desc()).select(['Date']).show()
print(type(df.orderBy(df['High'].desc()).select(['Date'])))
print(type(df.orderBy(df['High'].desc()).select(['Date']).head(1)))
df.orderBy(df['High'].desc()).select(['Date']).head(1)[0]['Date']

+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
|2015-01-08 00:00:00|
|2015-01-09 00:00:00|
|2015-01-12 00:00:00|
|2015-01-23 00:00:00|
|2015-01-26 00:00:00|
|2015-01-07 00:00:00|
|2015-01-14 00:00:00|
|2015-01-27 00:00:00|
|2015-01-22 00:00:00|
|2015-01-28 00:00:00|
|2014-11-28 00:00:00|
|2015-02-06 00:00:00|
|2015-01-15 00:00:00|
|2015-01-29 00:00:00|
|2015-01-20 00:00:00|
|2015-01-16 00:00:00|
|2014-12-31 00:00:00|
|2015-02-10 00:00:00|
|2015-01-30 00:00:00|
+-------------------+
only showing top 20 rows

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'list'>
Out[23]: datetime.datetime(2015, 1, 13, 0, 0)

In [0]:
#What is the mean of the Close column?

In [0]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [0]:
#What is the max and min of the Volume column?

In [0]:
from pyspark.sql.functions import min, max
df.select(max('Volume').alias("maximum_volume") ,min('Volume').alias("minimum_volume")).show()

+--------------+--------------+
|maximum_volume|minimum_volume|
+--------------+--------------+
|      80898100|       2094900|
+--------------+--------------+



In [0]:
#How many days was the Close lower than 60 dollars?

In [0]:
df.filter(df['Close'] < 60).count()

Out[27]: 81

In [0]:
6.	What percentage of the time was the High greater than 80 dollars ?

In [0]:
#meaning of above is (Number of Days High>80)/(Total Days in the dataset)
df.filter('High > 80').count() * 100/df.count()

Out[31]: 9.141494435612083

In [0]:
#7.	What is the Pearson correlation between High and Volume?

In [0]:

from pyspark.sql.functions import corr
df.select(corr(df['High'], df['Volume'])).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [0]:
#8.	What is the max High per year?

In [0]:
from pyspark.sql.functions import (dayofmonth, hour,dayofyear, month,year, weekofyear,format_number, date_format)
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').max()['Year', 'max(High)'].show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



In [0]:
#8.	What is the min High per year?

In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').min()['Year', 'min(High)'].show()

+----+------------------+
|Year|         min(High)|
+----+------------------+
|2015|         57.060001|
|2013| 68.66999799999999|
|2014|         73.099998|
|2012|             58.43|
|2016|61.490002000000004|
+----+------------------+



In [0]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [0]:
#no null columns in dataframe
a=df.columns
for i in a:
    print(str(i),df.where(df[i].isNull()).count())

Date 0
Open 0
High 0
Low 0
Close 0
Volume 0
Adj Close 0


In [0]:
#total number of records 
df.count()

Out[40]: 1258

In [0]:
df.distinct().count()
#AS Total record count equal to number of distinct record count there are no duplicate values in table

Out[41]: 1258

In [0]:
df.show(4)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 4 rows



In [0]:
#formating decimal and rounding to 2
from pyspark.sql.functions import format_number


df2=df.select(df["Date"],format_number(df['Open'].cast('float'),2).alias('Open'),
              format_number(df['High'].cast('float'),2).alias('High'),
              format_number(df['Low'].cast('float'),2).alias('Low'),
              format_number(df['Close'].cast('float'),2).alias('Close'),
              df['Volume'].cast('int').alias('Volume'),format_number(df["Adj Close"].cast('float'),2).alias('Adj Close')
             )
df2.show()

+-------------------+-----+-----+-----+-----+--------+---------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|
+-------------------+-----+-----+-----+-----+--------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|
|2012-01-06 00:00:00|59.42|59.45|58.87|59.00| 8069400|    51.46|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|
|2012-01-10 00:00:00|59.43|59.71|58.98|59.04| 6907300|    51.49|
|2012-01-11 00:00:00|59.06|59.53|59.04|59.40| 6365600|    51.81|
|2012-01-12 00:00:00|59.79|60.00|59.40|59.50| 7236400|    51.90|
|2012-01-13 00:00:00|59.18|59.61|59.01|59.54| 7729300|    51.93|
|2012-01-17 00:00:00|59.87|60.11|59.52|59.85| 8500000|    52.20|
|2012-01-18 00:00:00|59.79|60.03|59.65|60.01| 5911400|    52.34|
|2012-01-19 00:00:00|59.93|60.73|59.75|60.61| 9234600|    52.86|
|2012-01-20 00:00:00|60.7

In [0]:
#formating decimal and rounding to 2

df2.describe().show()


+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|72.35785373608894|72.83938791732918|71.91860095389515|72.38844992050863|8222093.481717011|67.23871224165353|
| stddev| 6.76809037021648|6.768186744823553|6.744075695823193|6.756859155425468|  4519780.8431556|6.722615952565601|
|    min|            56.39|            57.06|            56.30|            56.42|          2094900|            50.36|
|    max|            90.80|            90.97|            89.25|            90.47|         80898100|            84.91|
+-------+-----------------+-----------------+-----------

In [0]:
# average Close for each Calendar Month?
month_df = df.withColumn('Month', month(df['Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Close)'].show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



In [0]:
# average Low for each Calendar Month?
month_df = df.withColumn('Month', month(df['Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Low)'].show()

+-----+-----------------+
|Month|         avg(Low)|
+-----+-----------------+
|    1|70.90425712871289|
|    2|70.69360822680414|
|    3|71.31878489719628|
|    4|72.52104756190478|
|    5|71.85292466981134|
|    6|72.12198099056603|
|    7|74.00495360747668|
|    8|72.59472713636366|
|    9|71.76284345098038|
|   10|71.07990914545454|
|   11|71.54584138613862|
|   12|72.44481152830188|
+-----+-----------------+



In [0]:
# average High for each Calendar Month?
month_df = df.withColumn('Month', month(df['Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(High)'].show()

+-----+-----------------+
|Month|        avg(High)|
+-----+-----------------+
|    1|71.97009924752473|
|    2|71.72845387628868|
|    3|72.20289709345795|
|    4|73.45123863809525|
|    5|72.71783049056604|
|    6| 72.9262265471698|
|    7|74.77878527102803|
|    8|73.51781822727274|
|    9|72.60294148039216|
|   10|72.12963649090909|
|   11|72.52673264356439|
|   12|73.35566025471698|
+-----+-----------------+



In [0]:
#average open for each Calendar Month?
month_df = df.withColumn('Month', month(df['Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Open)'].show()

+-----+-----------------+
|Month|        avg(Open)|
+-----+-----------------+
|    1|71.40811884158416|
|    2|71.19123726804122|
|    3|71.69046716822429|
|    4|72.95723806666665|
|    5|72.24349083962262|
|    6| 72.5100938962264|
|    7|74.35756999065421|
|    8| 73.0720909181818|
|    9|72.17960762745093|
|   10|71.65472770909089|
|   11| 71.9740594059406|
|   12|72.87952850943395|
+-----+-----------------+



In [0]:
# average open for each Calendar Month?
month_df = df.withColumn('Month', month(df['Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Volume)'].show()

+-----+-----------------+
|Month|      avg(Volume)|
+-----+-----------------+
|    1|8761851.485148516|
|    2|9764147.422680411|
|    3|7721836.448598131|
|    4|7956230.476190476|
|    5|8632350.943396226|
|    6|8303756.603773585|
|    7|6841084.112149533|
|    8|7745198.181818182|
|    9|7194450.980392157|
|   10|9008785.454545455|
|   11|8921626.732673267|
|   12|7967959.433962264|
+-----+-----------------+



In [0]:
# average Adj Close for each Calendar Month?
month_df = df.withColumn('Month', month(df['Date']))
#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()
#Sort by month
month_df = month_df.orderBy('Month')
#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Adj Close)'].show()

+-----+-----------------+
|Month|   avg(Adj Close)|
+-----+-----------------+
|    1|65.56887865346533|
|    2|65.49688669072165|
|    3| 66.2763403084112|
|    4|67.44382143809523|
|    5|       67.0655795|
|    6|67.43827906603772|
|    7|69.10835440186914|
|    8|       68.1569289|
|    9|67.50560116666668|
|   10|66.78524237272727|
|   11|67.32305355445546|
|   12|68.46031040566042|
+-----+-----------------+



In [0]:
#From above we can say that walmart stock is performing consistantly over years from 2012 to 2016

In [0]:
from pyspark.sql.functions import year
#What is the average Close for each Calendar year?
year_df = df.withColumn('Year', year(df['Date']))
#Group by month and take average of all other columns
year_df = year_df.groupBy('Year').mean()
#Sort by month
year_df = year_df.orderBy('Year')
#Display only month and avg(Close), the desired columns
year_df['Year', 'avg(Close)'].show()

+----+-----------------+
|Year|       avg(Close)|
+----+-----------------+
|2012|67.21512003200003|
|2013|75.32051611904767|
|2014|77.32738088888888|
|2015|72.49111121825403|
|2016|69.54706346825398|
+----+-----------------+



In [0]:
# Open|    High|     Low|   Close|  Volume
from pyspark.sql.functions import year
#What is the average Close for each Calendar year?
year_df = df.withColumn('Year', year(df['Date']))
#Group by month and take average of all other columns
year_df = year_df.groupBy('Year').mean()
#Sort by month
year_df = year_df.orderBy('Year')
#Display only month and avg(Close), the desired columns
year_df['Year', 'avg(High)'].show()

+----+-----------------+
|Year|        avg(High)|
+----+-----------------+
|2012|67.60211992799995|
|2013|75.72940490079367|
|2014|77.74004000396819|
|2015| 73.0641670238095|
|2016| 70.0196429047619|
+----+-----------------+



In [0]:
from pyspark.sql.functions import year
#What is the average Close for each Calendar year?
year_df = df.withColumn('Year', year(df['Date']))
#Group by month and take average of all other columns
year_df = year_df.groupBy('Year').mean()
#Sort by month
year_df = year_df.orderBy('Year')
#Display only month and avg(Close), the desired columns
year_df['Year', 'avg(Low)'].show()

+----+-----------------+
|Year|         avg(Low)|
+----+-----------------+
|2012|66.78651992800005|
|2013|74.84305546428577|
|2014| 76.8644049047619|
|2015|72.03480166666671|
|2016|69.02349203174606|
+----+-----------------+



In [0]:
from pyspark.sql.functions import year
#What is the average Close for each Calendar year?
year_df = df.withColumn('Year', year(df['Date']))
#Group by month and take average of all other columns
year_df = year_df.groupBy('Year').mean()
#Sort by month
year_df = year_df.orderBy('Year')
#Display only month and avg(Close), the desired columns
year_df['Year', 'avg(Volume)'].show()

+----+-----------------+
|Year|      avg(Volume)|
+----+-----------------+
|2012|        9239015.2|
|2013|6951496.031746032|
|2014|6515612.301587301|
|2015|9040769.444444444|
|2016|9371645.238095239|
+----+-----------------+



In [0]:
from pyspark.sql.functions import year
#What is the average Close for each Calendar year?
year_df = df.withColumn('Year', year(df['Date']))
#Group by month and take average of all other columns
year_df = year_df.groupBy('Year').mean()
#Sort by month
year_df = year_df.orderBy('Year')
#Display only month and avg(Close), the desired columns
year_df['Year', 'avg(Adj Close)'].show()

+----+-----------------+
|Year|   avg(Adj Close)|
+----+-----------------+
|2012|59.38934891599998|
|2013|68.14717883730162|
|2014|71.70971223809524|
|2015|68.83142625396823|
|2016|68.05422865476187|
+----+-----------------+



In [0]:
from pyspark.sql.functions import (dayofmonth, hour,dayofyear, month,year, weekofyear,format_number, date_format)
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').max()['Year', 'max(High)'].show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').min()['Year', 'min(High)'].show()

+----+------------------+
|Year|         min(High)|
+----+------------------+
|2015|         57.060001|
|2013| 68.66999799999999|
|2014|         73.099998|
|2012|             58.43|
|2016|61.490002000000004|
+----+------------------+



In [0]:
#Open|    High|     Low|   Close|  Volume|Adj Close
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').max()['Year', 'max(Low)'].show()

+----+---------+
|Year| max(Low)|
+----+---------+
|2015|    89.25|
|2013|    80.82|
|2014|86.480003|
|2012|76.690002|
|2016|73.629997|
+----+---------+



In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').min()['Year', 'min(Low)'].show()

+----+---------+
|Year| min(Low)|
+----+---------+
|2015|56.299999|
|2013|67.720001|
|2014|72.269997|
|2012|    57.18|
|2016|60.200001|
+----+---------+



In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').max()['Year', 'max(Close)'].show()

+----+----------+
|Year|max(Close)|
+----+----------+
|2015| 90.470001|
|2013| 81.209999|
|2014| 87.540001|
|2012| 77.150002|
|2016| 74.300003|
+----+----------+



In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').min()['Year', 'min(Close)'].show()

+----+------------------+
|Year|        min(Close)|
+----+------------------+
|2015|         56.419998|
|2013|         68.300003|
|2014|         72.660004|
|2012|57.360001000000004|
|2016|             60.84|
+----+------------------+



In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').max()['Year', 'max(Adj Close)'].show()

+----+-----------------+
|Year|   max(Adj Close)|
+----+-----------------+
|2015|84.91421600000001|
|2013|        73.929868|
|2014|81.70768000000001|
|2012|        68.568371|
|2016|        73.233524|
+----+-----------------+



In [0]:
df_year = df.withColumn('Year', year(df['Date']))
df_year.groupBy('Year').min()['Year', 'min(Adj Close)'].show()

+----+------------------+
|Year|    min(Adj Close)|
+----+------------------+
|2015|53.975581000000005|
|2013|         61.039636|
|2014| 66.53139300000001|
|2012|         50.363689|
|2016|         58.691607|
+----+------------------+



In [0]:
#how many days were close lower than average value
df.filter(df['Close'] < 72.39).count()

Out[61]: 551

In [0]:
#how many days were high lower than average value
df.filter(df['High'] < 72.84).count()

Out[62]: 553

In [0]:
#how many days were high greater than average value
df.filter(df['High'] > 72.84).count()

Out[74]: 705

In [0]:
#how many days were low lower than average value
df.filter(df['Low'] < 71.92).count()

Out[70]: 547

In [0]:
#how many days were low lower than average value
df.filter(df['Low'] > 71.92).count()

Out[75]: 711

In [0]:
#As we see above number of days high and low fields of walmart stock is performing good when compared to average value

In [0]:
#how many days were volume lower than average value
df.filter(df['Volume'] < 8222093).count()

Out[72]: 823

In [0]:
#how many days were volume higher than average value
df.filter(df['Volume'] > 8222093).count()

Out[73]: 435

In [0]:
#What is the Pearson correlation between High and close?
#no linear relation between both columns
from pyspark.sql.functions import corr
df.select(corr(df['High'], df['Close'])).show()

+------------------+
| corr(High, Close)|
+------------------+
|0.9979708140519531|
+------------------+



In [0]:
#What is the Pearson correlation between High and Volume?
#no linear relation
from pyspark.sql.functions import corr
df.select(corr(df['High'], df['volume'])).show()

+-------------------+
| corr(High, volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [0]:
df.select(corr(df['Low'], df['volume'])).show()

+--------------------+
|   corr(Low, volume)|
+--------------------+
|-0.38206212516421373|
+--------------------+



In [0]:
df.select(corr(df['Adj Close'], df['volume'])).show()

+-----------------------+
|corr(Adj Close, volume)|
+-----------------------+
|   -0.31867529308373727|
+-----------------------+



In [0]:
df.select(corr(df['Close'], df['volume'])).show()

+--------------------+
| corr(Close, volume)|
+--------------------+
|-0.36864129499157394|
+--------------------+



In [0]:
#'close','Adj Close','open'
df.select(corr(df['High'], df['Low'])).show()

+-----------------+
|  corr(High, Low)|
+-----------------+
|0.997555844579623|
+-----------------+



In [0]:
df.select(corr(df['Adj Close'], df['Low'])).show()

+--------------------+
|corr(Adj Close, Low)|
+--------------------+
|  0.9337522333432656|
+--------------------+



In [0]:
df.select(corr(df['open'], df['Low'])).show()

+------------------+
|   corr(open, Low)|
+------------------+
|0.9978038357411027|
+------------------+



In [0]:
#how volume impacted by difference between high and low
df4 = df2.withColumn('Range', df2['High']-df2['Low'])
print(df4.corr('Range','Volume'))


0.604848899889576


In [0]:
#how volume is impacted by change in cost
df5 = df2.withColumn('Change', df2['Close']-df2['Open'])
print(df5.corr('Volume','Change'))



-0.14262642523929162
